<a href="https://colab.research.google.com/github/glitter127/Analysis/blob/main/TwitterClustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tweepy


In [2]:
import tweepy

# Add your credentials here
api_key = "your_api_key"
api_key_secret = "your_api_key_secret"
access_token = "your_access_token"
access_token_secret = "your_access_token_secret"
BEARER_TOKEN = "your_BEARER_TOKEN"

# Authenticate
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# Verify authentication
try:
    user = api.verify_credentials()
    print(f"Authenticated as {user.screen_name}")
except Exception as e:
    print("Authentication failed:", e)


Authentication failed: 401 Unauthorized
89 - Invalid or expired token.


In [ ]:
import tweepy


# Authenticate using Twitter API v2
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Fetch live tweets using v2 API
def fetch_tweets_v2(query, count=10):
    tweets = []
    response = client.search_recent_tweets(query=query, max_results=count, tweet_fields=["text"])
    for tweet in response.data:
        tweets.append(tweet.text)
    return tweets

# Fetch tweets
tweets = fetch_tweets_v2("Python", count=10)
print(tweets)


In [ ]:
import tweepy
import pandas as pd
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Download NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Replace with your own Twitter API credentials
BEARER_TOKEN = BEARER_TOKEN

# Authenticate with Twitter API v2
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Function to fetch live tweets
def fetch_tweets_v2(query, count=10):
    tweets = []
    response = client.search_recent_tweets(query=query, max_results=count, tweet_fields=["text"])
    if response.data:
        for tweet in response.data:
            tweets.append(tweet.text)
    return tweets

# Function to clean tweets
def clean_tweet(text):
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r"\@\w+|\#", "", text)  # Remove mentions and hashtags
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    text = " ".join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

# Fetch and clean tweets
tweets = fetch_tweets_v2("Python", count=10)
cleaned_tweets = [clean_tweet(tweet) for tweet in tweets]

# Convert text into numerical form using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(cleaned_tweets)

# Apply K-Means clustering
num_clusters = 3  # Choose the number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

# Assign cluster labels to tweets
df = pd.DataFrame({"Tweet": cleaned_tweets, "Cluster": kmeans.labels_})

# Print clustered tweets
for i in range(num_clusters):
    print(f"\n🔹 Cluster {i}:")
    print(df[df["Cluster"] == i]["Tweet"].tolist())
